In [2]:
import pandas as pd
import os
import json

In [2]:
def genDesc(filename):
    try:
        bb_file = pd.read_csv(filename, sep=' ', header=None)
        counts = bb_file[8].value_counts()
        text = 'A remote sensing containing '
        for label, count in counts.items():
            text = text + str(count) + ' ' + label.replace('-', ' ') + 's, '
        text = text[:-2] + "."
        return text
    except:
        return 'A satellite image.'


In [3]:
train_descriptions = []
val_descriptions = []
train_dir = '../datasets/DOTAv2/labels/DOTA-v2.0_train_original_format/'
val_dir = '../datasets/DOTAv2/labels/DOTA-v2.0_val_original_format/'
train_files = [f for f in os.listdir(train_dir)]
val_files = [f for f in os.listdir(val_dir)]

for f in train_files:
    src_path = os.path.join(train_dir, f)
    desc = genDesc(src_path)
    dict = {
        'filename': f,
        'description': desc
    }
    train_descriptions.append(dict)

for f in val_files:
    src_path = os.path.join(val_dir, f)
    desc = genDesc(src_path)
    dict = {
        'filename': f,
        'description': desc
    }
    val_descriptions.append(dict)

In [4]:
with open("../datasets/DOTAv2/descriptions/train.json", "w") as outfile:
    json.dump(train_descriptions, outfile)
with open("../datasets/DOTAv2/descriptions/val.json", "w") as outfile:
    json.dump(val_descriptions, outfile)

In [14]:
bb_file = pd.read_csv('../datasets/DOTAv2/labels/DOTA-v2.0_train_original_format/P0002.txt', sep=' ', header=None)

In [15]:
bb_file = bb_file.drop(9, axis=1)

In [22]:
bb_file['x_centroid'] = bb_file[[0,2,4,6]].sum(axis=1)/4

In [23]:
bb_file['y_centroid'] = bb_file[[1,3,5,7]].sum(axis=1)/4

In [24]:
bb_file

,0,1,2,3,4,5,6,7,8,x_centroid,y_centroid
0,506.0,201.0,481.0,199.0,477.0,83.0,501.0,83.0,large-vehicle,491.25,141.50
1,642.0,1468.0,654.0,1467.0,653.0,1503.0,639.0,1503.0,small-vehicle,647.00,1485.25
2,1169.0,805.0,1191.0,794.0,1253.0,885.0,1225.0,897.0,large-vehicle,1209.50,845.25
3,1209.0,812.0,1231.0,799.0,1290.0,891.0,1265.0,904.0,large-vehicle,1248.75,851.50
4,1280.0,816.0,1300.0,805.0,1360.0,896.0,1334.0,909.0,large-vehicle,1318.50,856.50
...,...,...,...,...,...,...,...,...,...,...,...
283,1430.0,1123.0,1449.0,1136.0,1417.0,1188.0,1396.0,1174.0,large-vehicle,1423.00,1155.25
284,1460.0,1127.0,1482.0,1139.0,1447.0,1193.0,1427.0,1179.0,large-vehicle,1454.00,1159.50
285,1495.0,1126.0,1515.0,1137.0,1480.0,1189.0,1461.0,1177.0,large-vehicle,1487.75,1157.25
286,1527.0,1128.0,1547.0,1139.0,1514.0,1193.0,1494.0,1181.0,large-vehicle,1520.50,1160.25
